# Physics AI - Full Colab Run

This notebook runs the full project flow in Google Colab, not only smoke checks.

## 1) Clone repo and install dependencies

In [ ]:
import subprocess
from pathlib import Path

REPO_URL = "https://github.com/santanu-p/Autonomous-Theoretical-Physics-Research-Engine.git"
REPO_DIR = Path(REPO_URL.rstrip('/').split('/')[-1].replace('.git', ''))

if not REPO_DIR.exists():
    subprocess.run(["git", "clone", REPO_URL], check=True)

%cd {REPO_DIR}
!pip -q install -U pip
!pip -q install -e ".[dev,llm,hpc]"

## 2) Optional LLM setup (NVIDIA)

Skip if you want deterministic fallback proposals only.

In [ ]:
import os

# Uncomment and set your key for LLM proposals:
# os.environ["NVIDIA_API_KEY"] = "YOUR_NVIDIA_KEY"

os.environ.setdefault("PHYSAI_LLM_BASE_URL", "https://integrate.api.nvidia.com/v1")
os.environ.setdefault("PHYSAI_LLM_MODEL", "meta/glm-5")
print("LLM enabled:", "NVIDIA_API_KEY" in os.environ)

## 3) Run test suite

In [ ]:
!pytest -q

## 4) Run full deterministic pipeline

In [ ]:
!physai theory validate examples/theories/einstein_hilbert.yaml
!physai derive eom examples/theories/einstein_hilbert.yaml --run-id colab_full
!physai derive background examples/theories/einstein_hilbert.yaml --ansatz static_spherical --run-id colab_full
!physai derive perturb artifacts/colab_full/background/background.json --run-id colab_full
!physai run qnm artifacts/colab_full/perturb/perturbation.json --method shooting --run-id colab_full
!physai run qnm artifacts/colab_full/perturb/perturbation.json --method wkb --run-id colab_full
!physai run qnm artifacts/colab_full/perturb/perturbation.json --method spectral --run-id colab_full
!physai run scan examples/scan.yaml
!physai novelty score colab_full
!physai paper build colab_full --title "Autonomous Physics AI Full Run"

## 5) Run full autonomous campaign (proposal -> paper)

In [ ]:
import os
import time
from pathlib import Path
import yaml

src = Path("examples/campaigns/default.yaml")
dst = Path("examples/campaigns/colab_full.yaml")
cfg = yaml.safe_load(src.read_text())
cfg["run_id"] = f"colab_campaign_{int(time.time())}"
cfg["enable_llm"] = "NVIDIA_API_KEY" in os.environ
dst.write_text(yaml.safe_dump(cfg, sort_keys=False))
print("Campaign file:", dst)
print("run_id:", cfg["run_id"])
print("enable_llm:", cfg["enable_llm"])

In [ ]:
!physai orchestrate autonomous --campaign examples/campaigns/colab_full.yaml

## 6) Inspect and download artifacts

In [ ]:
!find artifacts -maxdepth 6 -type f | head -n 200

In [ ]:
!zip -r artifacts_full_run.zip artifacts > /dev/null
from google.colab import files
files.download('artifacts_full_run.zip')